In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# Read in the obesity activity data.

obesity_activity = pd.read_csv(
    '../data/03_health_data/IHME_USA_OBESITY_PHYSICAL_ACTIVITY_2001_2011.csv', 
    dtype = {'merged_fips': object})

In [5]:
# Fix the FIPS code.
obesity_activity['fips'] = obesity_activity['merged_fips'].str.zfill(5)
obesity_activity

,fips,merged_fips,State,County,Sex,Outcome,Prevalence 2001 (%),Prevalence 2001 lower bound (%),Prevalence 2001 upper bound (%),Prevalence 2002 (%),Prevalence 2002 lower bound (%),Prevalence 2002 upper bound (%),Prevalence 2003 (%),Prevalence 2003 lower bound (%),Prevalence 2003 upper bound (%),Prevalence 2004 (%),Prevalence 2004 lower bound (%),Prevalence 2004 upper bound (%),Prevalence 2005 (%),Prevalence 2005 lower bound (%),Prevalence 2005 upper bound (%),Prevalence 2006 (%),Prevalence 2006 lower bound (%),Prevalence 2006 upper bound (%),Prevalence 2007 (%),Prevalence 2007 lower bound (%),Prevalence 2007 upper bound (%),Prevalence 2008 (%),Prevalence 2008 lower bound (%),Prevalence 2008 upper bound (%),Prevalence 2009 (%),Prevalence 2009 lower bound (%),Prevalence 2009 upper bound (%),Prevalence 2010 (%),Prevalence 2010 lower bound (%),Prevalence 2010 upper bound (%),Prevalence 2011 (%),Prevalence 2011 lower bound (%),Prevalence 2011 upper bound (%),Difference 2001 to 2009,Difference 2001 to 2009 lower bound,Difference 2001 to 2009 upper bound
0,01001,1001,Alabama,"Autauga, AL",Male,Any PA,76.72,71.45,80.94,77.30,72.42,81.32,77.63,73.02,81.67,77.03,72.46,81.18,77.21,72.76,81.27,76.72,71.90,80.69,77.32,72.74,81.46,77.32,72.76,81.24,76.73,71.89,80.81,76.37,71.06,80.87,76.05,70.21,81.02,0.01,-6.70,6.75
1,01001,1001,Alabama,"Autauga, AL",Female,Any PA,68.21,62.94,73.12,68.79,63.92,73.25,69.20,64.59,73.48,67.39,62.16,72.19,67.60,62.79,72.12,67.26,62.19,72.30,67.39,62.12,71.45,67.09,62.50,71.09,68.77,64.50,72.86,68.74,64.19,72.93,68.75,63.69,73.12,0.55,-6.32,6.91
2,01003,1003,Alabama,"Baldwin, AL",Male,Any PA,73.90,69.75,78.34,74.41,70.39,78.57,74.87,70.95,78.84,75.67,71.86,79.16,75.10,71.19,78.58,77.02,73.60,80.47,78.02,74.58,81.04,79.51,76.53,82.36,78.33,75.56,81.16,78.04,74.96,81.10,77.69,74.17,81.13,4.43,-0.88,9.39
3,01003,1003,Alabama,"Baldwin, AL",Female,Any PA,71.84,67.69,75.63,72.66,68.91,76.16,73.46,69.75,76.90,74.50,71.07,77.53,75.62,72.44,78.65,76.59,73.79,79.25,75.57,72.52,78.29,73.52,70.60,75.94,72.87,70.36,75.43,72.69,69.93,75.29,72.56,69.43,75.45,1.03,-3.59,6.10
4,01005,1005,Alabama,"Barbour, AL",Male,Any PA,65.61,59.40,71.07,66.21,60.24,71.66,66.80,60.90,72.30,66.99,61.23,72.73,65.43,59.55,71.01,63.98,58.31,69.15,65.63,59.70,70.98,66.55,61.06,72.10,66.66,61.61,71.81,66.23,60.61,71.73,65.75,59.24,72.07,1.05,-6.94,9.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18853,56043,56043,Wyoming,"Washakie, WY",Female,Sufficient PA,54.69,48.40,61.33,NaN,NaN,NaN,55.87,50.04,61.77,NaN,NaN,NaN,55.52,50.35,60.60,NaN,NaN,NaN,55.07,49.28,60.87,NaN,NaN,NaN,55.33,49.09,61.54,NaN,NaN,NaN,56.22,49.05,63.55,0.64,-7.90,9.78
18854,56045,56045,Wyoming,"Weston, WY",Male,Sufficient PA,52.94,45.63,60.21,NaN,NaN,NaN,54.41,48.05,60.86,NaN,NaN,NaN,55.86,50.04,61.09,NaN,NaN,NaN,59.19,53.55,64.61,NaN,NaN,NaN,58.27,51.36,64.80,NaN,NaN,NaN,56.78,46.38,66.07,5.33,-4.63,14.84
18855,56045,56045,Wyoming,"Weston, WY",Female,Sufficient PA,53.86,47.82,60.00,NaN,NaN,NaN,55.19,49.76,60.43,NaN,NaN,NaN,54.98,49.62,60.20,NaN,NaN,NaN,54.78,48.93,60.23,NaN,NaN,NaN,55.11,48.59,61.44,NaN,NaN,NaN,56.03,48.32,63.67,1.25,-7.66,10.66
18856,02911,2911,Alaska,"Denali, AK",Male,Sufficient PA,58.37,51.68,64.33,NaN,NaN,NaN,59.96,53.93,65.24,NaN,NaN,NaN,56.64,50.32,62.38,NaN,NaN,NaN,59.21,52.84,64.90,NaN,NaN,NaN,56.80,49.97,63.17,NaN,NaN,NaN,55.77,46.65,64.23,-1.57,-11.73,8.08


In [14]:
# Pick off just the males obesity info for 2011.
obesity_males = obesity_activity[(obesity_activity['Outcome'] == 'Obesity') & (obesity_activity['Sex']=='Male')].copy()
obesity_males.set_index('fips', inplace = True)
obesity_males = obesity_males[['Prevalence 2011 lower bound (%)', 'Prevalence 2011 upper bound (%)']]
obesity_males.columns = ['male_2011_lower_bound_percent', 'male_2011_upper_bound_percent']

# Pick off just the females obesity info for 2011.
obesity_females = obesity_activity[(obesity_activity['Outcome'] == 'Obesity') & (obesity_activity['Sex']=='Female')].copy()
obesity_females.set_index('fips', inplace = True)
obesity_females = obesity_females[['Prevalence 2011 lower bound (%)', 'Prevalence 2011 upper bound (%)']]
obesity_females.columns = ['female_2011_lower_bound_percent', 'female_2011_upper_bound_percent']

obesity = pd.concat([obesity_females, obesity_males], axis = 1)
obesity

,female_2011_lower_bound_percent,female_2011_upper_bound_percent,male_2011_lower_bound_percent,male_2011_upper_bound_percent
fips,,,,
01001,34.06,43.23,32.17,42.71
01003,29.93,36.74,29.93,38.63
01005,45.95,55.80,36.57,47.55
01007,39.55,49.36,35.44,46.52
01009,36.28,46.80,33.36,44.05
...,...,...,...,...
56039,16.71,22.53,18.62,25.12
56041,31.01,38.42,32.01,40.74
56043,28.77,38.99,27.34,37.71


Need to drop the `merged_fips` where it is 02911. This is not a FIPS code in the ACS data that we read in elsewhere. We'll do that below.

In [15]:
demo_df = pd.read_csv(
    '../data/cleaned/final_demo.csv', 
    dtype = {'fips' : object}
)

#demo_df
demo_df.set_index('fips', inplace = True)
demo_df

,total_pop_age_sex,median_age,sex_ratio_males,under_18_percent,over_18_percent,over_65_percent,county,state,total_hh,median_hh_income,mean_hh_income,total_families,median_family_income,mean_family_income,tot_pop_race_ethnicity,tot_white,tot_black_af_am,tot_aminalnat,tot_asian,tot_hawopi,total_pop_poverty,pop_below_poverty,percent_below_poverty,smoking_ban_2010,asian_2019,avg_family_size_2019,black_2019,hispanic_2019,household_has_broadband_2019,household_has_computer_2019,household_has_smartphone_2019,households_2019,households_speak_limited_english_2019,housing_mobile_homes_2019,hs_grad_2019,median_household_income_2019,median_individual_income_2019,native_2019,other_single_race_2019,pac_isl_2019,persons_per_household_2019,pop_2019,unemployment_rate_2019,uninsured_2019,veterans_2019,white_2019,white_not_hispanic_2019,Metro2013,rural_urban_continuum_code_2013,retirement_destination_2015_update,metro_adjacent_2013
fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01001,55380.0,38.2,94.7,23.8,76.2,15.0,Autauga County,Alabama,21397.0,58731.0,75326.0,15076.0,71103.0,87094.0,55380.0,42527.0,10538.0,140.0,573.0,26.0,54922.0,8340.0,15.2,none,1.0,3.09,19.0,2.8,80.6,73.0,78.4,21397.0,0.7,26.7,88.5,58731.0,29725.0,0.3,0.7,0.0,2.56,55380.0,3.5,7.1,12.6,76.8,74.6,2.0,1.0,1.0,0.0
01003,212830.0,43.0,94.7,21.7,78.3,20.0,Baldwin County,Alabama,80930.0,58320.0,80986.0,53467.0,75850.0,97991.0,212830.0,183471.0,19718.0,1645.0,1969.0,9.0,209618.0,21704.0,10.4,none,0.9,3.24,9.3,4.6,81.8,76.3,81.7,80930.0,1.2,24.8,90.8,58320.0,29802.0,0.8,1.1,0.0,2.59,212830.0,4.0,8.9,11.8,86.2,83.1,3.0,1.0,1.0,0.0
01005,25361.0,40.4,112.4,20.9,79.1,18.6,Barbour County,Alabama,9345.0,32525.0,47068.0,6187.0,41704.0,56374.0,25361.0,11869.0,12066.0,82.0,134.0,1.0,22417.0,6875.0,30.7,partial,0.5,3.01,47.6,4.4,60.5,51.9,64.2,9345.0,1.6,39.1,73.2,32525.0,17963.0,0.3,3.6,0.0,2.41,25361.0,9.4,11.3,6.6,46.8,45.8,6.0,0.0,0.0,1.0
01007,22493.0,40.9,117.5,20.6,79.4,15.9,Bibb County,Alabama,6891.0,47542.0,60182.0,4789.0,57891.0,69316.0,22493.0,17272.0,5014.0,30.0,27.0,0.0,20632.0,3740.0,18.1,none,0.1,3.74,22.3,2.6,69.2,54.7,66.6,6891.0,0.6,25.6,79.1,47542.0,21958.0,0.1,0.0,0.0,2.99,22493.0,7.0,10.7,8.0,76.8,74.5,1.0,1.0,0.0,0.0
01009,57681.0,40.7,97.6,23.2,76.8,17.9,Blount County,Alabama,20847.0,49358.0,65639.0,14874.0,62295.0,76547.0,57681.0,55062.0,928.0,46.0,212.0,25.0,57108.0,7739.0,13.6,none,0.4,3.33,1.6,9.3,73.0,63.5,70.1,20847.0,1.8,21.2,80.5,49358.0,26976.0,0.1,0.9,0.0,2.74,57681.0,3.1,10.8,7.7,95.5,86.9,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,43521.0,35.3,106.8,26.5,73.5,11.4,Sweetwater County,Wyoming,15523.0,74843.0,85346.0,10794.0,87906.0,97459.0,43521.0,40642.0,502.0,508.0,340.0,19.0,42958.0,4921.0,11.5,none,0.8,3.32,1.2,15.9,84.0,81.9,83.4,15523.0,1.9,23.8,92.8,74843.0,36512.0,1.2,1.2,0.0,2.77,43521.0,5.7,11.3,8.6,93.4,79.6,5.0,0.0,0.0,0.0
56039,23280.0,39.3,112.5,19.1,80.9,14.0,Teton County,Wyoming,9019.0,84678.0,132531.0,5190.0,110667.0,172024.0,23280.0,20785.0,290.0,77.0,291.0,29.0,23188.0,1426.0,6.1,partial,1.3,3.01,1.2,15.0,87.1,90.1,85.3,9019.0,5.7,39.6,95.2,84678.0,40914.0,0.3,7.1,0.1,2.47,23280.0,0.7,12.7,5.3,89.3,81.3,7.0,0.0,0.0,0.0
56041,20479.0,35.8,103.3,29.2,70.8,13.0,Uinta County,Wyoming,7597.0,63403.0,74938.0,5313.0,77725.0,86265.0,20479.0,19132.0,23.0,145.0,32.0,0.0,20259.0,2298.0,11.3,none,0.2,3.23,0.1,9.1,89.5,80.3,84.8,7597.0,0.6,22.9,92.7,63403.0,29557.0,0.7,2.1,0.0,2.66,20479.0,5.5,11.2,7.4,93.4,87.5,7.0,0.0,0.0,0.0


In [16]:
# Checking to see what comes up when looking on `Denali` on the demographic dataframe and it's not FIPS 02911. So we'll drop it.
demo_df[demo_df['county'].str.contains('Denali')]

,total_pop_age_sex,median_age,sex_ratio_males,under_18_percent,over_18_percent,over_65_percent,county,state,total_hh,median_hh_income,mean_hh_income,total_families,median_family_income,mean_family_income,tot_pop_race_ethnicity,tot_white,tot_black_af_am,tot_aminalnat,tot_asian,tot_hawopi,total_pop_poverty,pop_below_poverty,percent_below_poverty,smoking_ban_2010,asian_2019,avg_family_size_2019,black_2019,hispanic_2019,household_has_broadband_2019,household_has_computer_2019,household_has_smartphone_2019,households_2019,households_speak_limited_english_2019,housing_mobile_homes_2019,hs_grad_2019,median_household_income_2019,median_individual_income_2019,native_2019,other_single_race_2019,pac_isl_2019,persons_per_household_2019,pop_2019,unemployment_rate_2019,uninsured_2019,veterans_2019,white_2019,white_not_hispanic_2019,Metro2013,rural_urban_continuum_code_2013,retirement_destination_2015_update,metro_adjacent_2013
fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
02068,2246.0,42.1,114.9,11.6,88.4,9.3,Denali Borough,Alaska,613.0,81719.0,85694.0,321.0,104375.0,107659.0,2246.0,1892.0,26.0,49.0,58.0,0.0,2170.0,354.0,16.3,none,2.6,3.12,1.2,0.3,89.9,91.5,79.8,613.0,0.0,18.4,98.5,81719.0,26630.0,2.2,0.2,0.0,2.32,2246.0,1.1,18.5,6.9,84.2,84.2,8.0,0.0,0.0,1.0


In [17]:
obesity

,female_2011_lower_bound_percent,female_2011_upper_bound_percent,male_2011_lower_bound_percent,male_2011_upper_bound_percent
fips,,,,
01001,34.06,43.23,32.17,42.71
01003,29.93,36.74,29.93,38.63
01005,45.95,55.80,36.57,47.55
01007,39.55,49.36,35.44,46.52
01009,36.28,46.80,33.36,44.05
...,...,...,...,...
56039,16.71,22.53,18.62,25.12
56041,31.01,38.42,32.01,40.74
56043,28.77,38.99,27.34,37.71


In [19]:
obesity.drop(index = '02911', inplace = True)

In [22]:
demo_df

,total_pop_age_sex,median_age,sex_ratio_males,under_18_percent,over_18_percent,over_65_percent,county,state,total_hh,median_hh_income,mean_hh_income,total_families,median_family_income,mean_family_income,tot_pop_race_ethnicity,tot_white,tot_black_af_am,tot_aminalnat,tot_asian,tot_hawopi,total_pop_poverty,pop_below_poverty,percent_below_poverty,smoking_ban_2010,asian_2019,avg_family_size_2019,black_2019,hispanic_2019,household_has_broadband_2019,household_has_computer_2019,household_has_smartphone_2019,households_2019,households_speak_limited_english_2019,housing_mobile_homes_2019,hs_grad_2019,median_household_income_2019,median_individual_income_2019,native_2019,other_single_race_2019,pac_isl_2019,persons_per_household_2019,pop_2019,unemployment_rate_2019,uninsured_2019,veterans_2019,white_2019,white_not_hispanic_2019,Metro2013,rural_urban_continuum_code_2013,retirement_destination_2015_update,metro_adjacent_2013
fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01001,55380.0,38.2,94.7,23.8,76.2,15.0,Autauga County,Alabama,21397.0,58731.0,75326.0,15076.0,71103.0,87094.0,55380.0,42527.0,10538.0,140.0,573.0,26.0,54922.0,8340.0,15.2,none,1.0,3.09,19.0,2.8,80.6,73.0,78.4,21397.0,0.7,26.7,88.5,58731.0,29725.0,0.3,0.7,0.0,2.56,55380.0,3.5,7.1,12.6,76.8,74.6,2.0,1.0,1.0,0.0
01003,212830.0,43.0,94.7,21.7,78.3,20.0,Baldwin County,Alabama,80930.0,58320.0,80986.0,53467.0,75850.0,97991.0,212830.0,183471.0,19718.0,1645.0,1969.0,9.0,209618.0,21704.0,10.4,none,0.9,3.24,9.3,4.6,81.8,76.3,81.7,80930.0,1.2,24.8,90.8,58320.0,29802.0,0.8,1.1,0.0,2.59,212830.0,4.0,8.9,11.8,86.2,83.1,3.0,1.0,1.0,0.0
01005,25361.0,40.4,112.4,20.9,79.1,18.6,Barbour County,Alabama,9345.0,32525.0,47068.0,6187.0,41704.0,56374.0,25361.0,11869.0,12066.0,82.0,134.0,1.0,22417.0,6875.0,30.7,partial,0.5,3.01,47.6,4.4,60.5,51.9,64.2,9345.0,1.6,39.1,73.2,32525.0,17963.0,0.3,3.6,0.0,2.41,25361.0,9.4,11.3,6.6,46.8,45.8,6.0,0.0,0.0,1.0
01007,22493.0,40.9,117.5,20.6,79.4,15.9,Bibb County,Alabama,6891.0,47542.0,60182.0,4789.0,57891.0,69316.0,22493.0,17272.0,5014.0,30.0,27.0,0.0,20632.0,3740.0,18.1,none,0.1,3.74,22.3,2.6,69.2,54.7,66.6,6891.0,0.6,25.6,79.1,47542.0,21958.0,0.1,0.0,0.0,2.99,22493.0,7.0,10.7,8.0,76.8,74.5,1.0,1.0,0.0,0.0
01009,57681.0,40.7,97.6,23.2,76.8,17.9,Blount County,Alabama,20847.0,49358.0,65639.0,14874.0,62295.0,76547.0,57681.0,55062.0,928.0,46.0,212.0,25.0,57108.0,7739.0,13.6,none,0.4,3.33,1.6,9.3,73.0,63.5,70.1,20847.0,1.8,21.2,80.5,49358.0,26976.0,0.1,0.9,0.0,2.74,57681.0,3.1,10.8,7.7,95.5,86.9,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,43521.0,35.3,106.8,26.5,73.5,11.4,Sweetwater County,Wyoming,15523.0,74843.0,85346.0,10794.0,87906.0,97459.0,43521.0,40642.0,502.0,508.0,340.0,19.0,42958.0,4921.0,11.5,none,0.8,3.32,1.2,15.9,84.0,81.9,83.4,15523.0,1.9,23.8,92.8,74843.0,36512.0,1.2,1.2,0.0,2.77,43521.0,5.7,11.3,8.6,93.4,79.6,5.0,0.0,0.0,0.0
56039,23280.0,39.3,112.5,19.1,80.9,14.0,Teton County,Wyoming,9019.0,84678.0,132531.0,5190.0,110667.0,172024.0,23280.0,20785.0,290.0,77.0,291.0,29.0,23188.0,1426.0,6.1,partial,1.3,3.01,1.2,15.0,87.1,90.1,85.3,9019.0,5.7,39.6,95.2,84678.0,40914.0,0.3,7.1,0.1,2.47,23280.0,0.7,12.7,5.3,89.3,81.3,7.0,0.0,0.0,0.0
56041,20479.0,35.8,103.3,29.2,70.8,13.0,Uinta County,Wyoming,7597.0,63403.0,74938.0,5313.0,77725.0,86265.0,20479.0,19132.0,23.0,145.0,32.0,0.0,20259.0,2298.0,11.3,none,0.2,3.23,0.1,9.1,89.5,80.3,84.8,7597.0,0.6,22.9,92.7,63403.0,29557.0,0.7,2.1,0.0,2.66,20479.0,5.5,11.2,7.4,93.4,87.5,7.0,0.0,0.0,0.0


In [23]:
obesity

,female_2011_lower_bound_percent,female_2011_upper_bound_percent,male_2011_lower_bound_percent,male_2011_upper_bound_percent
fips,,,,
01001,34.06,43.23,32.17,42.71
01003,29.93,36.74,29.93,38.63
01005,45.95,55.80,36.57,47.55
01007,39.55,49.36,35.44,46.52
01009,36.28,46.80,33.36,44.05
...,...,...,...,...
56037,32.73,39.31,31.38,39.03
56039,16.71,22.53,18.62,25.12
56041,31.01,38.42,32.01,40.74


In [26]:
pd.merge(left = demo_df, right = obesity, on='fips')

,total_pop_age_sex,median_age,sex_ratio_males,under_18_percent,over_18_percent,over_65_percent,county,state,total_hh,median_hh_income,mean_hh_income,total_families,median_family_income,mean_family_income,tot_pop_race_ethnicity,tot_white,tot_black_af_am,tot_aminalnat,tot_asian,tot_hawopi,total_pop_poverty,pop_below_poverty,percent_below_poverty,smoking_ban_2010,asian_2019,avg_family_size_2019,black_2019,hispanic_2019,household_has_broadband_2019,household_has_computer_2019,household_has_smartphone_2019,households_2019,households_speak_limited_english_2019,housing_mobile_homes_2019,hs_grad_2019,median_household_income_2019,median_individual_income_2019,native_2019,other_single_race_2019,pac_isl_2019,persons_per_household_2019,pop_2019,unemployment_rate_2019,uninsured_2019,veterans_2019,white_2019,white_not_hispanic_2019,Metro2013,rural_urban_continuum_code_2013,retirement_destination_2015_update,metro_adjacent_2013,female_2011_lower_bound_percent,female_2011_upper_bound_percent,male_2011_lower_bound_percent,male_2011_upper_bound_percent
fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01001,55380.0,38.2,94.7,23.8,76.2,15.0,Autauga County,Alabama,21397.0,58731.0,75326.0,15076.0,71103.0,87094.0,55380.0,42527.0,10538.0,140.0,573.0,26.0,54922.0,8340.0,15.2,none,1.0,3.09,19.0,2.8,80.6,73.0,78.4,21397.0,0.7,26.7,88.5,58731.0,29725.0,0.3,0.7,0.0,2.56,55380.0,3.5,7.1,12.6,76.8,74.6,2.0,1.0,1.0,0.0,34.06,43.23,32.17,42.71
01003,212830.0,43.0,94.7,21.7,78.3,20.0,Baldwin County,Alabama,80930.0,58320.0,80986.0,53467.0,75850.0,97991.0,212830.0,183471.0,19718.0,1645.0,1969.0,9.0,209618.0,21704.0,10.4,none,0.9,3.24,9.3,4.6,81.8,76.3,81.7,80930.0,1.2,24.8,90.8,58320.0,29802.0,0.8,1.1,0.0,2.59,212830.0,4.0,8.9,11.8,86.2,83.1,3.0,1.0,1.0,0.0,29.93,36.74,29.93,38.63
01005,25361.0,40.4,112.4,20.9,79.1,18.6,Barbour County,Alabama,9345.0,32525.0,47068.0,6187.0,41704.0,56374.0,25361.0,11869.0,12066.0,82.0,134.0,1.0,22417.0,6875.0,30.7,partial,0.5,3.01,47.6,4.4,60.5,51.9,64.2,9345.0,1.6,39.1,73.2,32525.0,17963.0,0.3,3.6,0.0,2.41,25361.0,9.4,11.3,6.6,46.8,45.8,6.0,0.0,0.0,1.0,45.95,55.80,36.57,47.55
01007,22493.0,40.9,117.5,20.6,79.4,15.9,Bibb County,Alabama,6891.0,47542.0,60182.0,4789.0,57891.0,69316.0,22493.0,17272.0,5014.0,30.0,27.0,0.0,20632.0,3740.0,18.1,none,0.1,3.74,22.3,2.6,69.2,54.7,66.6,6891.0,0.6,25.6,79.1,47542.0,21958.0,0.1,0.0,0.0,2.99,22493.0,7.0,10.7,8.0,76.8,74.5,1.0,1.0,0.0,0.0,39.55,49.36,35.44,46.52
01009,57681.0,40.7,97.6,23.2,76.8,17.9,Blount County,Alabama,20847.0,49358.0,65639.0,14874.0,62295.0,76547.0,57681.0,55062.0,928.0,46.0,212.0,25.0,57108.0,7739.0,13.6,none,0.4,3.33,1.6,9.3,73.0,63.5,70.1,20847.0,1.8,21.2,80.5,49358.0,26976.0,0.1,0.9,0.0,2.74,57681.0,3.1,10.8,7.7,95.5,86.9,1.0,1.0,0.0,0.0,36.28,46.80,33.36,44.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,43521.0,35.3,106.8,26.5,73.5,11.4,Sweetwater County,Wyoming,15523.0,74843.0,85346.0,10794.0,87906.0,97459.0,43521.0,40642.0,502.0,508.0,340.0,19.0,42958.0,4921.0,11.5,none,0.8,3.32,1.2,15.9,84.0,81.9,83.4,15523.0,1.9,23.8,92.8,74843.0,36512.0,1.2,1.2,0.0,2.77,43521.0,5.7,11.3,8.6,93.4,79.6,5.0,0.0,0.0,0.0,32.73,39.31,31.38,39.03
56039,23280.0,39.3,112.5,19.1,80.9,14.0,Teton County,Wyoming,9019.0,84678.0,132531.0,5190.0,110667.0,172024.0,23280.0,20785.0,290.0,77.0,291.0,29.0,23188.0,1426.0,6.1,partial,1.3,3.01,1.2,15.0,87.1,90.1,85.3,9019.0,5.7,39.6,95.2,84678.0,40914.0,0.3,7.1,0.1,2.47,23280.0,0.7,12.7,5.3,89.3,81.3,7.0,0.0,0.0,0.0,16.71,22.53,18.62,25.12
56041,20479.0,35.8,103.3,29.2,70.8,13.0,Uinta County,Wyoming,7597.0,63403.0,74938.0,5313.0,77725.0,86265.0,20479.0,19132.0,23.0,145.0,32.0,0.0,20259.0,2298.0,11.3,none,0.2,3.23,0.1,9.1,89.5,80.3,84.8,7597.0,0.6,22.9,92.7,63403.0,29557.0,0.7,2.1,0.0,2.66,20479.0,5.5,11.2,7.4,93.4,87.5,7.0,0.0,0.0,0.0,31.01,38.42,32.01,40.74


In [25]:
pd.concat([demo_df, obesity], axis =1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects